# 1. Radar
Por: Camila Lopes ([cclopes.me](https://cclopes.me/))  


Neste módulo, iremos aprender a lidar com dados de radares meteorológicos usando a linguagem Python, passando por leitura, visualização e processamento.


In [ ]:
# Importando as bibliotecas necessárias
import matplotlib.pyplot as plt

import pyart
import wradlib as wrl

from read_mira_radar import read_mira
from read_brazil_radar_py3 import read_rainbow_hdf5
from read_sipam_cappis import read_sipam_cappi



## 1.2. Visualizando dados de radar

Agora que sabemos como ler diferentes formatos de dados de radar (seção 1.1), vamos aprender a visualizar esses dados com as bibliotecas Py-ART e wradlib.


### 1.2.1. Py-ART

Como explicado anteriormente, o Py-ART possui 4 classes de variáveis com dados de radar: [`Radar`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar), [`Grid`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Grid.html#pyart.core.Grid), [`HorizontalWindProfile`](https://arm-doe.github.io/pyart/API/generated/pyart.core.HorizontalWindProfile.html#pyart.core.HorizontalWindProfile) ou [`RadarSpectra`](https://arm-doe.github.io/pyart/API/generated/pyart.core.RadarSpectra.html#pyart.core.RadarSpectra). Para cada classe existem classes e funções correspondentes para a visualização desses dados, que estão no módulo [`pyart.graph`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.html). Essas classes e suas respectivas funções serão explicadas a seguir.

Para todas as classes e funções de visualização de dados, a mesma estrutura é utilizada:

```{python}
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.[classe]([variável de radar])
# Abrindo a figura
fig = plt.figure()
# Gerando o plot
display.[função de plot](field=[campo da variável], [...])
```

Note que a variável de radar não é usada diretamente na função que plota os dados. Os parâmetros das funções de plot podem variar bastante, mas é sempre necessário especificar o campo (refletividade, velocidade doppler, etc) a ser plotado.

O código abaixo abre diferentes tipos de dados de radar para testarmos cada uma das classes.


In [ ]:
# Lendo arquivos
dados_pp = pyart.io.read("../dados/radar/PP/PPR161225194610.RAWK318")
dados_sr = pyart.aux_io.read_gamic("../dados/radar/SR/SRO-250--2017-03-14--18-30-22.mvol")
dados_sipam = pyart.aux_io.read_gamic("../dados/radar/SIPAM/RADL08061720140103111200.HDF5")
dados_mira = read_mira("../dados/radar/MIRA_CAMPINA/20200306_0100.mmclx")[0]
dados_fcth = read_rainbow_hdf5("../dados/radar/FCTH/PNOVA2-20170314182730.HDF5")
dados_xpol = read_rainbow_hdf5("../dados/radar/XPOL_CMP/117BRX-20171115215006.HDF5")
dados_sipam_cappi = read_sipam_cappi("../dados/radar/SIPAM/sbmn_cappi_20140103_120016.nc")


#### [`RadarDisplay`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.html#pyart.graph.RadarDisplay)

A classe `RadarDisplay` plota dados do tipo `Radar` sem georreferenciamento. As funções de plot disponíveis são:

- [`plot(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot.html#pyart.graph.RadarDisplay.plot): similar à função `pyart.io.read()` mostrada na seção 1.1.2, essa função identifica o tipo de escaneamento (variável `scan_type`) e plota o campo de radar em uma projeção correspondente à ele (chamando a função `plot_ppi()`, `plot_rhi()` ou `plot_vpt()` explicadas abaixo). Além do campo (`field`), é necessário fornecer a elevação (`sweep`) ou será plotada a primeira elevação por padrão. É possível definir diversos parâmetros como limites de valores (`vmin` e `vmax`), escala de cores (`cmap`), legendas de título (`title`), eixos (`axislabels`) e escala de cores (`colorbar_label`), entre outros;

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_sr)
# Abrindo a figura
fig = plt.figure(figsize=(5, 4))
# Gerando o plot
display.plot(field="reflectivity", sweep=0)


In [ ]:
# Como exemplo do plot de RHI, podemos gerar um "pseudo-RHI" dos dados de PPI, selecionando o azimute de interesse
# Isso não é mais necessário pois a função plot_azimuth_to_rhi() faz isso diretamente
cross_sec = pyart.util.cross_section_ppi(dados_sr, [0])

# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(cross_sec)
# Abrindo a figura
fig = plt.figure(figsize=(8, 4))
# Gerando o plot
# Como o cross_sec só tem um azimute, não é preciso definir sweep
display.plot(field="reflectivity", vmin=0, vmax=70)
# Arrumando o eixo y
plt.ylim((0, 20))

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_mira)
# Abrindo a figura
fig = plt.figure(figsize=(8, 4))
# Gerando o plot
# Como é do tipo VPT, não é necessário definir sweep
display.plot(field="Ze")

- [`plot_ppi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_ppi.html#pyart.graph.RadarDisplay.plot_ppi): plota o campo de radar (`field`) do tipo PPI em uma dada elevação (`sweep`);

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_xpol)
# Abrindo a figura
fig = plt.figure(figsize=(5, 4))
# Gerando o plot
display.plot(field="reflectivity", sweep=1)
# Adicionando o contorno de 250 km de raio e uma grade
display.plot_range_ring(250)
display.plot_grid_lines()

- [`plot_ray(field, ray)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_ray.html#pyart.graph.RadarDisplay.plot_ray): plota um único feixe (`ray`) do tipo PPI de um campo (`field`) em uma dada elevação (`sweep`);

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_xpol)
# Abrindo a figura
fig = plt.figure(figsize=(6, 4))
# Gerando o plot
display.plot_ray(field="reflectivity", ray=310)

- [`plot_azimuth_to_rhi(field, target_azimuth)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_azimuth_to_rhi.html#pyart.graph.RadarDisplay.plot_azimuth_to_rhi): plota um pseudo-RHI de dados de PPI de um campo (`field`) em um determinado azimute (`target_azimuth`);

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_xpol)
# Abrindo a figura
fig = plt.figure(figsize=(8, 4))
# Gerando o plot
display.plot_azimuth_to_rhi(field="reflectivity", target_azimuth=310, vmin=0, vmax=70)
# Arrumando o eixo y
plt.ylim((0, 20))
# Adicionando uma grade
display.plot_grid_lines()


- [`plot_rhi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_rhi.html#pyart.graph.RadarDisplay.plot_rhi): plota um perfil vertical de dados de RHI de um campo (`field`) em um determinado azimute (`sweep`);


In [ ]:
# Como exemplo do plot de RHI, podemos gerar um "pseudo-RHI" dos dados de PPI, selecionando o azimute de interesse
# Isso não é mais necessário pois a função plot_azimuth_to_rhi() faz isso diretamente
cross_sec = pyart.util.cross_section_ppi(dados_xpol, [300, 310, 320])

# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(cross_sec)
# Abrindo a figura
fig = plt.figure(figsize=(8, 4))
# Gerando o plot
display.plot_rhi(field="reflectivity", sweep=2, vmin=0, vmax=70)
# Arrumando o eixo y e adicionando uma grade
plt.ylim((0, 20))
display.plot_grid_lines()

- [`plot_vpt(field)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_vpt.html#pyart.graph.RadarDisplay.plot_vpt): plota um perfil de apontamento vertical de um dado campo (`field`).

In [ ]:
# Gerando um display da classe a partir da variável de radar
display = pyart.graph.RadarDisplay(dados_mira)
# Abrindo a figura
fig = plt.figure(figsize=(8, 4))
# Gerando o plot
display.plot_vpt(field="VEL", vmin=-10, vmax=10, cmap="pyart_BuDRd18")

#### [`RadarMapDisplay`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.html#pyart.graph.RadarMapDisplay) ([`RadarMapDisplayBasemap`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplayBasemap.html#pyart.graph.RadarMapDisplayBasemap))

A classe `RadarMapDisplay` plota dados do tipo `Radar` com georreferenciamento. A partir da versão 1.8, a classe "padrão" usa a bibliotexa [`cartopy`](https://scitools.org.uk/cartopy/docs/latest/) para a projeção em mapas, mas essa classe foi feita originalmente com a biblioteca [`basemap`](https://matplotlib.org/basemap/) (que está atualmente descontinuada) e suas funções correspondentes ainda estão disponíveis na classe `RadarMapDisplayBasemap`. Isso implica em algumas diferenças nos parâmetros utilizados nas funções de cada classe, que devem ser consultadas na [página da classe](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplayBasemap.html#pyart.graph.RadarMapDisplayBasemap). *Para ficar mais breve, apenas a classe `RadarMapDisplay` será descrita abaixo*.  

As funções de plot disponíveis são:

- [`plot(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot.html#pyart.graph.RadarMapDisplay.plot): similar à função `pyart.io.read()` mostrada na seção 1.1.2, essa função identifica o tipo de escaneamento (variável `scan_type`) e plota o campo de radar em uma projeção correspondente à ele (chamando a função `plot_ppi()`, `plot_rhi()` ou `plot_vpt()` explicadas abaixo). Além do campo (`field`), é necessário fornecer a elevação (`sweep`) ou será plotada a primeira elevação por padrão. É possível definir diversos parâmetros como limites de valores (`vmin` e `vmax`), escala de cores (`cmap`), legendas de título (`title`), eixos (`axislabels`) e escala de cores (`colorbar_label`), entre outros;

- [`plot_ppi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_ppi.html#pyart.graph.RadarMapDisplay.plot_ppi): plota o campo de radar (`field`) do tipo PPI em uma dada elevação (`sweep`);

- [`plot_ppi_map(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_ppi_map.html#pyart.graph.RadarMapDisplay.plot_ppi_map): plota o campo de radar (`field`) do tipo PPI em uma dada elevação (`sweep`);

- [`plot_ray(field, ray)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_ray.html#pyart.graph.RadarMapDisplay.plot_ray): plota um único feixe (`ray`) do tipo PPI de um campo (`field`) em uma dada elevação (`sweep`);

- [`plot_azimuth_to_rhi(field, target_azimuth)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_azimuth_to_rhi.html#pyart.graph.RadarMapDisplay.plot_azimuth_to_rhi): plota um pseudo-RHI de dados de PPI de um campo (`field`) em um determinado azimute (`target_azimuth`);

- [`plot_rhi(field, sweep=0)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_rhi.html#pyart.graph.RadarMapDisplay.plot_rhi): plota um perfil vertical de dados de RHI de um campo (`field`) em um determinado azimute (`sweep`);


- [`plot_vpt(field)`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_vpt.html#pyart.graph.RadarMapDisplay.plot_vpt): plota um perfil de apontamento vertical de um dado campo (`field`).

#### [`GridMapDisplay`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.GridMapDisplay.html#pyart.graph.GridMapDisplay) ([`GridMapDisplayBasemap`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.GridMapDisplayBasemap.html#pyart.graph.GridMapDisplayBasemap))

#### [`AirborneRadarDisplay`](https://arm-doe.github.io/pyart/API/generated/pyart.graph.AirborneRadarDisplay.html#pyart.graph.AirborneRadarDisplay)

### 1.2.2. wradlib